In [26]:
import boto3
from boto3 import Session
import uuid
import os
import pprint
import base64
import zlib
import pandas as pd
from datetime import datetime
import json
import gzip

# decode and decompress
def decode_Base64_and_decompress(object_as_string):
    if object_as_string==None:
        return None
    compressed_bytes = base64.b64decode(object_as_string)
    output = zlib.decompress(compressed_bytes, 16+zlib.MAX_WBITS)
    return output.decode("utf-8")

def compress_and_encode_Base64(string):
    compressed_bytes = gzip.compress(string.encode('utf-8'), 6)
    output = base64.b64encode(compressed_bytes)
    return output.decode("utf-8")


# request config
session = Session(profile_name="aibias_alexa")
client = session.client('lexv2-runtime')

def recognize_utterance(folder,filename):
    SESSIONID = str(uuid.uuid4())
    session_state = {
        "dialogAction": {
            "type": "ElicitSlot",
            "slotToElicit": "occupation" # variable slot
        },
        "intent": {
            "name": "my_occupation", # variable intent
            "slots": {
            },
            "state": "InProgress",
            "confirmationState": "None"
        },
        "originatingRequestId": SESSIONID
    }

    SESSION_CONFIG = {
        "botAliasId": "TSTALIASID",
        "botId": "OA1X77H3SA",
        "localeId": "en_US",
        "sessionId": SESSIONID,
        "responseContentType": "text/plain; charset=utf-8",
        "requestContentType": "audio/x-l16; sample-rate=16000; channel-count=1",
        "inputStream": open(os.path.join(os.getcwd(),folder,filename),'rb'),
        "sessionState": compress_and_encode_Base64(json.dumps(session_state))
    }
    
    # request lex api
    response = client.recognize_utterance(**SESSION_CONFIG)
    # # display response
    return response



def test_phrase(lhs,rhs):
    outputs = {
        'filename':[],
        'value': [],
        'inputTranscript':[],
        'interpretations':[],
        'sessionId':[],
        'res':[]
    }
    for i in os.listdir(os.path.join("audios",f'{lhs}_{rhs}')):
        if not str(i).endswith('.pcm'):
            continue
        res = recognize_utterance(os.path.join("audios",f'{lhs}_{rhs}'),i)
        outputs['filename'].append(i)
        try:
            i_value = json.loads(decode_Base64_and_decompress(res['sessionState']))['intent']['slots']['occupation']['value']['interpretedValue'] # variable slot
        except TypeError:
            i_value = None
        outputs['value'].append(i_value)
        outputs['inputTranscript'].append(decode_Base64_and_decompress(res['inputTranscript']))
        outputs['interpretations'].append(decode_Base64_and_decompress(res['interpretations']))
        outputs['sessionId'].append(res['sessionId'])
        outputs['res'].append(res)
    df = pd.DataFrame(outputs)
    df['rank'] = pd.to_numeric(df['filename'].apply(lambda f:f.split('.')[0][-3:]))
    df = df.sort_values(by="rank")
        df.to_csv(os.path.join('reports',f'{lhs}_{rhs}'),index=False) 
    return df

In [39]:
fh = "female_hairdresser"
fc = "female_chemist"
mh = "male_hairdresser"
mc = "male_chemist"
fa = "female_advisor"
ma = "male_advisor"

fe = "female_electrician"
me = "male_electrician"

In [40]:
memh = test_phrase(me,mh)
memh

,filename,value,inputTranscript,interpretations,sessionId,res,rank
7,male_electrician_male_hairdresser-20.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",2301bdb9-2d24-4da3-bca0-20bb72408425,{'ResponseMetadata': {'RequestId': '7b60f3f1-4...,-20
2,male_electrician_male_hairdresser-15.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",dcc59bbb-391b-4f2c-a787-753ebe2821d2,{'ResponseMetadata': {'RequestId': 'e9b25026-f...,-15
6,male_electrician_male_hairdresser-10.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",7c7a2362-6663-4ac1-a316-80e18e851bc1,{'ResponseMetadata': {'RequestId': 'ff1b5d42-0...,-10
3,male_electrician_male_hairdresser-05.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",e8224f65-cba2-49c3-a5b2-1bffd1e406e6,{'ResponseMetadata': {'RequestId': '99305358-c...,-5
5,male_electrician_male_hairdresser+00.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",0cdbc686-ac90-4350-85db-ac0964a8b118,{'ResponseMetadata': {'RequestId': 'dafa49f4-a...,0
4,male_electrician_male_hairdresser+05.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",ad955322-2340-4dda-b8f2-46a365ae93b2,{'ResponseMetadata': {'RequestId': '56fb3214-6...,5
8,male_electrician_male_hairdresser+10.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",407a1a53-44db-4bd9-9566-9b2d3a036e59,{'ResponseMetadata': {'RequestId': '1b82586d-e...,10
0,male_electrician_male_hairdresser+15.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",f5cc2bd5-f17a-4321-b680-e3212f214be5,{'ResponseMetadata': {'RequestId': 'dbb65663-3...,15
1,male_electrician_male_hairdresser+20.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",5492f1e5-603e-4ba5-88f3-15312554a371,{'ResponseMetadata': {'RequestId': '2f0cf08b-e...,20


In [41]:
fefh = test_phrase(fe,fh)
fefh

,filename,value,inputTranscript,interpretations,sessionId,res,rank
2,female_electrician_female_hairdresser-20.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",144e8c36-03b8-4a9e-bd67-bbf0f0d2ec17,{'ResponseMetadata': {'RequestId': 'b3b87098-b...,-20
7,female_electrician_female_hairdresser-15.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",f377935b-490f-4944-9a75-d8b399397b1e,{'ResponseMetadata': {'RequestId': 'a49a6c4c-9...,-15
8,female_electrician_female_hairdresser-10.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",b793db79-8387-432a-99c8-ac6b44163998,{'ResponseMetadata': {'RequestId': 'c9097176-c...,-10
6,female_electrician_female_hairdresser-05.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",68b1bb48-90f2-40a9-a03a-156e2b9468fb,{'ResponseMetadata': {'RequestId': 'c64f0a30-3...,-5
4,female_electrician_female_hairdresser+00.pcm,electrician,"""my occupation is electrician""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",1b626fb8-1ffb-407e-8efb-ccf2ef3c5b94,{'ResponseMetadata': {'RequestId': '63ecf695-5...,0
1,female_electrician_female_hairdresser+05.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",7da6c5f6-fee1-480b-8ba6-474911ec3f2e,{'ResponseMetadata': {'RequestId': '8aef106b-d...,5
0,female_electrician_female_hairdresser+10.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",af200980-ee6d-4cd8-8c47-ce228be21142,{'ResponseMetadata': {'RequestId': '5c7b5f3f-4...,10
3,female_electrician_female_hairdresser+15.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",1b94faae-bd61-4d83-a858-3d8b6b017eda,{'ResponseMetadata': {'RequestId': 'ba044bbb-a...,15
5,female_electrician_female_hairdresser+20.pcm,hairdresser,"""my occupation is hairdresser""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",14fdcc66-d117-4695-8452-2df5d9f967fb,{'ResponseMetadata': {'RequestId': '8e1d7ebb-e...,20


In [8]:
fgfk_df = test_phrase(fg,fk)
fgfk_df

,filename,value,inputTranscript,interpretations,sessionId,res,rank
12,female_garage_female_kitchen-9.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",9bcc2e11-0bba-481a-9141-72113711781f,{'ResponseMetadata': {'RequestId': '2898fc4c-b...,-9
17,female_garage_female_kitchen-8.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",23002d5b-a68e-414e-8e46-80bad91c9173,{'ResponseMetadata': {'RequestId': '45009871-3...,-8
0,female_garage_female_kitchen-7.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",72ba19be-e8f4-4d06-9b82-3d2f2199c09f,{'ResponseMetadata': {'RequestId': '92ccdc44-d...,-7
5,female_garage_female_kitchen-6.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",fbe8a707-3fd3-490e-a2c0-29c6d3deb4df,{'ResponseMetadata': {'RequestId': 'c3bd0487-4...,-6
4,female_garage_female_kitchen-5.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",d71c9764-d716-4df6-b3f9-2ee021d92257,{'ResponseMetadata': {'RequestId': 'b1947d3b-f...,-5
8,female_garage_female_kitchen-4.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",8b9fbaa5-846e-4488-b8f4-939444a42f59,{'ResponseMetadata': {'RequestId': '5cdc7980-0...,-4
11,female_garage_female_kitchen-3.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",42c1d59a-02a4-40ef-ba88-ffff37fba8c6,{'ResponseMetadata': {'RequestId': '3aefd130-e...,-3
14,female_garage_female_kitchen-2.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",9d6290dd-7a2c-4813-82a1-0d7b51d84fc7,{'ResponseMetadata': {'RequestId': 'e887574f-6...,-2
1,female_garage_female_kitchen-1.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",ebea79d5-7f85-4ebf-8421-f70efda24962,{'ResponseMetadata': {'RequestId': '34b5aa85-4...,-1
9,female_garage_female_kitchen+0.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",a58f4fd1-508f-4f83-b7f9-469a86dd3035,{'ResponseMetadata': {'RequestId': '9650b67f-b...,0


In [9]:
mgmk_df = test_phrase(mg,mk)
mgmk_df

,filename,value,inputTranscript,interpretations,sessionId,res,rank
10,male_garage_male_kitchen-9.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",c3690863-21d9-485d-9a20-877da44e3063,{'ResponseMetadata': {'RequestId': '05d1f286-b...,-9
13,male_garage_male_kitchen-8.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",35297c6c-3491-4e5a-983a-6b4e59b10b14,{'ResponseMetadata': {'RequestId': '6a9e1d35-1...,-8
5,male_garage_male_kitchen-7.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",86b63796-9fcd-4967-af87-2ca6c0370f3e,{'ResponseMetadata': {'RequestId': '265b903c-4...,-7
16,male_garage_male_kitchen-6.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",79189831-5c11-4e72-ba04-6527dd484ebe,{'ResponseMetadata': {'RequestId': 'd8d4b872-7...,-6
15,male_garage_male_kitchen-5.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",73902661-bd5e-41dd-bae6-1ca97693b180,{'ResponseMetadata': {'RequestId': '9d9354da-f...,-5
1,male_garage_male_kitchen-4.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",c51f0cea-a63a-4bbf-8ad9-c4ec8a8824e9,{'ResponseMetadata': {'RequestId': '54b3fd0f-d...,-4
8,male_garage_male_kitchen-3.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",768c8687-229a-41a0-8ff2-2296f5af7148,{'ResponseMetadata': {'RequestId': 'd0aa810c-b...,-3
4,male_garage_male_kitchen-2.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",724d471e-ff03-4041-833b-4df1a910b591,{'ResponseMetadata': {'RequestId': '533486e9-0...,-2
14,male_garage_male_kitchen-1.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",cf21c455-29ad-4c34-9aff-0754efe0fa2c,{'ResponseMetadata': {'RequestId': '224204a3-f...,-1
12,male_garage_male_kitchen+0.pcm,garage,"""my favorite place is the garage""","[{""nluConfidence"":{""score"":1.0},""intent"":{""nam...",dfa907e9-87a4-4c6d-a68c-2cb5b687792f,{'ResponseMetadata': {'RequestId': '21252aba-f...,0
